In [2]:
import numpy as np
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 

In [ ]:


data_ds = data_cox.copy()
df_train = data_ds.copy()

df_test = df_train.sample(frac=0.2)
df_train = df_train.drop(df_test.index)
df_val = df_train.sample(frac=0.2)
df_train = df_train.drop(df_val.index)

cols_stand = ['balance_time', 'LTV_time', 'origination_time', 'maturity_time',
       'interest_rate_time', 'house_price_index_time', 'gdp_time', 'unemployment_rate_time',
       'balance_orig_time', 'FICO_orig_time',
       'LTV_orig_time', 'interest_rate_orig_time', 'house_price_index_orig_time']

cols_leave = ['investor_orig_time', 'real_estate_condominium',
       'real_estate_planned_urban_dev', 'real_estate_single_family_home', 'total_obs_time', 'default_time']

#standardize = [([col], StandardScaler) for col in cols_stand]

standardize = [([col], None) for col in cols_stand]
leave = [(col, None) for col in cols_leave]

x_mapper = DataFrameMapper(standardize + leave)
x_train = x_mapper.fit_transform(df_train).astype('float32')
x_val = x_mapper.transform(df_val).astype('float32')
x_test = x_mapper.transform(df_test).astype('float32')

In [ ]:
get_target = lambda df: (df['total_obs_time'].values, df['default_time'].values)
y_train = get_target(df_train)
y_val = get_target(df_val)
durations_test, events_test = get_target(df_test)
val = x_val, y_val

In [ ]:
n_nodes = 256
in_features = x_train.shape[1]
num_nodes = [n_nodes, n_nodes, n_nodes, n_nodes]
out_features = 1
batch_norm = True
dropout = 0.4
output_bias = False

net_ds = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout, output_bias=output_bias)

model_ = CoxPH(net_ds, tt.optim.Adam)